<a href="https://colab.research.google.com/github/k-stacke/cmiv-ai-course/blob/master/ChatGPT_Prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Promting with ChatGPT

## Problem

How can we use Large Language Models (ChatGPT in specific) to solve text-based problems such as text summarization and data mining from unstructured text?

## Imports and data loading

In [ ]:
#!pip install --upgrade openai

import os
import io
import requests
import openai
from openai import AzureOpenAI

In [ ]:
# NOTE: update to other info once we have a token specific for the AI course

response = requests.get('CHANGE ME')
config = response.json()

In [ ]:
# Set up the clinent to access the OpenAI API

client = AzureOpenAI(
    api_version=config['OPENAI_API_VERSION'],
    azure_endpoint=config['AZURE_OPENAI_ENDPOINT'],
    api_key=config['AZURE_OPENAI_API_KEY'])

CHATGPT_ENGINE_NAME = config['engine_name-chatgpt']
print("Using engine name", CHATGPT_ENGINE_NAME, "and API version", openai.api_version)

In [ ]:
# Helper function for calling the model
def get_completion(prompt, model=CHATGPT_ENGINE_NAME, temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

# Writing prompt

In [ ]:
text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""

prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence. 
```{text}```
"""


response = get_completion(prompt)
print(response)

In [ ]:
prompt = f"""
Write the following using only emoticons.

'''Nu ska vi prata ChatGPT. Det blir roligt. Men oroa er inte, snart är det kaffe!'
"""

response = get_completion(prompt)
print(response)

## Text summarization

In [ ]:
text = """ CT -  CT ABDOMEN AND PELVIS ENHANCED -  24-Feb-2020, 12:45 PM
&nbsp;  
&nbsp;  
INDICATION:  Rule out diverticulitis   .&nbsp;  
&nbsp;  
TECHNIQUE: Volumetric image acquisition of the       abdomen and pelvis    with   intravenous   contrast.  Coronal  and sagittal  reformats were performed post-acquisition.&nbsp;  
&nbsp;  
COMPARISON:  CT renal colic study from 10/04/16 &nbsp;  
&nbsp;  
FINDINGS:&nbsp;  
&nbsp;  
    There are a few scattered hypoattenuating lesions in the liver likely representing cysts. Cysts were described on the prior CT examination. The&nbsp;  
&nbsp;  
There is no evidence for colonic diverticulosis or colitis. The appendix is identified and is normal.&nbsp;  
&nbsp;  
There is a trace amount of pelvic free fluid identified. There is mild hazy stranding in the left adnexal region which may be inflammatory. A definite adnexal mass or cyst is not seen but a recently ruptured cyst could potentially demonstrate this finding. Please note the gynecologic structures are not optimally assessed by CT. &nbsp;  
&nbsp;  
The    remainder of the    intra-abdominal structures show no gross   abnormality   .Lung bases are grossly clear. &nbsp;  
&nbsp;  
No aggressive bony abnormality is identified.&nbsp;  
&nbsp;  
OPINION: &nbsp;  
&nbsp;  
   No evidence for diverticulosis/diverticulitis/colitis. Appendix is normal.&nbsp;  
&nbsp;  
Mild inflammatory change in the left adnexal region which may be secondary to recent ruptured cyst. Ultrasound follow-up could be performed. &nbsp;  
&nbsp;  
If you have received this report in error, please return by Fax to Health Information Management at Trillium Health Partners at 905-848-7677. If you don't have access to fax, or have other questions, please call 905-848-7580 ext. 2172."""


In [ ]:
prompt = f"""Format the text below:  
 {text}
 """

response = get_completion(prompt)
print(response)

In [ ]:
instruction = "Explain the report below to me as if I where 10 years old"

prompt = f"""{instruction}: 
 {text}
 """
response = get_completion(prompt)
print(response)

## Task

Try and change the style of the explaination. Some example prompts to get the imagination going:
+ `Explain the report below to me as if I was the patient/a high school student/75 years old`
+ `Explain the report below to me giving the most inportant parts as a bullent point list`
+ `Explain the report below to me by first defining all medical terms in a list, then give a short summarization of the conclusions with 1-3 sentences.`

## Extract structured information from unstructured report

Large language models (LLMs) are good at extracting information from unstructured data.

In [ ]:
instruction = """Extract information about all nodules found in a JSON fomat with the categories size and location. 
Change metric values to imperial.
If no nodules are found, return empty list."""

text = "This is an example radiologist report. One nodule found, 2.5x2.5cm located in the lower-left part of the liver."

prompt = f"""{instruction}: 
 {text}
 """

response = get_completion(prompt)
print(response)

In [ ]:
instruction = "Extract information about all nodules found in a JSON fomat with the categories size and location. If no nodules are found, return empty brackets."

text = "This is an example radiologist report. Normal exam, no nodules found. The nodule found in the previous exam (of size 2.5cmx2.5cm) is not visible."

prompt = f"""{instruction}: 
 {text}
 """

response = get_completion(prompt)
print(response)

## Task

Play around with the instruction and the text. 
1. Keep the prompt constant and change the text. How does it behave? When does it fail? What type of information does/cab it extract?
2. Keep the text constant and change the prompt. Can you fix so that it outputs what you want? Can you get it to answer in another language? Can you improve the quality of the output by adding more/clearer instructions?

## Beware of hallucinations

ChatGPT and other LLMs has a tendency to output so called hallucinations. It has no knowledge of what is true or false, and may therefore output answers to questions/tasks sounding confident but is actually incorrect or making things up.

In [ ]:
instruction = "Grade the nodule according to the Swedish Society of Radiology guidelines of liver nodules."

text = """This is an example radiologist report. One nodule found, 2.5x2.5cm located in the lower-left part of the liver."""


prompt = f"""{instruction}: 
 {text}
 """

response = get_completion(prompt)
print(response)

## Task

Play with the text and prompt and see if you can get the model to output inconsistensies or incorrect information. See if you can change the prompt to mitigate this behaviour. 

You can add reference text to the instructions and ask the model to only use these when solving the task.